In [3]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from pathlib import Path

# ---------------------------
# Mann–Kendall Test with Sen’s Slope (α = 0.05)
# ---------------------------
def mann_kendall_with_sen(series, alpha=0.05):
    """
    Mann–Kendall test with Sen’s slope estimation.
    Evaluates trend significance at 95% confidence (alpha=0.05).
    """
    x = np.array(series)
    x = x[~np.isnan(x)]
    n = len(x)

    if n < 3:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, "Not Enough Data"

    # Mann–Kendall S statistic
    S = 0
    for k in range(n - 1):
        for j in range(k + 1, n):
            S += np.sign(x[j] - x[k])

    # Variance of S (no tie correction)
    var_S = (n * (n - 1) * (2 * n + 5)) / 18
    if var_S == 0:
        return S, var_S, np.nan, np.nan, np.nan, np.nan, "Variance is Zero"

    # Z-statistic
    if S > 0:
        Z = (S - 1) / np.sqrt(var_S)
    elif S < 0:
        Z = (S + 1) / np.sqrt(var_S)
    else:
        Z = 0

    # Two-tailed p-value
    p = 2 * (1 - norm.cdf(abs(Z)))
    Z_crit = norm.ppf(1 - alpha / 2)

    # Determine trend
    if abs(Z) > Z_crit:
        trend = "Increasing" if Z > 0 else "Decreasing"
    else:
        trend = "No Significant Trend"

    # Sen’s slope
    slopes = [(x[j] - x[i]) / (j - i) for i in range(n - 1) for j in range(i + 1, n)]
    sen_slope = np.median(slopes)
    intercept = np.median(x - sen_slope * np.arange(n))

    return S, var_S, Z, p, sen_slope, intercept, trend


# ---------------------------
# Process Yearly Averages File
# ---------------------------
file = Path(r"D:\climate change\yearly_averages.xlsx")
sheets = ["Rainfall_yearly", "Tmax_yearly", "Tmin_yearly"]  # Modify if needed

results = []
print(f"Starting Mann–Kendall (95% confidence) analysis on: {file.name}")

for sheet in sheets:
    print(f"\n📂 Processing Sheet: {sheet}")
    try:
        # Read yearly average data
        df = pd.read_excel(file, sheet_name=sheet, index_col=0)
        df = df.drop(columns=["Lon", "Lat"], errors="ignore")

        # Ensure datetime index
        df.index = pd.to_datetime(df.index.astype(str), format="%Y", errors="coerce")

        for station in df.columns:
            S, varS, Z, p, slope, intercept, trend = mann_kendall_with_sen(df[station])

            results.append({
                "Sheet": sheet,
                "Station": station,
                "S": round(S, 2),
                "Var(S)": round(varS, 2),
                "Z": round(Z, 2),
                "p-value": round(p, 2),
                "Sen's Slope (per year)": round(slope, 2),
                "Intercept": round(intercept, 2),
                "Trend (95%)": trend
            })

        print(f"  ✅ Processed {len(df.columns)} stations for {len(df)} years.")

    except Exception as e:
        print(f"  ⚠️ Error processing sheet '{sheet}': {e}")


# ---------------------------
# Save Output
# ---------------------------
output = Path(r"D:\climate change\MK_SenSlope_results_yearly_95.csv")
pd.DataFrame(results).to_csv(output, index=False)

print(f"\n✅ Mann–Kendall + Sen’s slope (95% confidence) results saved to:\n{output}")


Starting Mann–Kendall (95% confidence) analysis on: yearly_averages.xlsx

📂 Processing Sheet: Rainfall_yearly
  ✅ Processed 23 stations for 48 years.

📂 Processing Sheet: Tmax_yearly
  ✅ Processed 23 stations for 48 years.

📂 Processing Sheet: Tmin_yearly
  ✅ Processed 23 stations for 48 years.

✅ Mann–Kendall + Sen’s slope (95% confidence) results saved to:
D:\climate change\MK_SenSlope_results_yearly_95.csv
